In [1]:
import itertools
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
import anndata

import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
import seaborn as sns

from scroutines import basicu
# from scroutines import powerplots

In [2]:
sns.set_context('talk')

# base line

In [3]:
f = "../data/cheng21_cell_scrna/organized/P28NR.h5ad"

adata = anndata.read(f, backed='r')
genes = adata.var.index.values
genes


array(['4933401J01Rik', 'Gm26206', 'Xkr4', ..., 'CAAA01064564.1',
       'Vmn2r122', 'CAAA01147332.1'], dtype=object)

In [4]:
f = "../results/MERFISH_gene_panel_current_Feb28.csv"
df = pd.read_csv(f)

cnddts = df['gene_name_data'].values
unq, cnts = np.unique(cnddts, return_counts=True)
print(len(cnddts), unq.shape, unq[cnts>1], [g for g in cnddts if g not in genes])

df

832 (832,) [] []


,gene_name_vizgen,gene_name_data,why included,source,Annot1,Annot2,Annot3
0,Matn2,Matn2,L2/3 subtypes,Cheng22_Cell,A>C>B,screened,NaN
1,Egfem1,Egfem1,L2/3 subtypes;NRvsDR_DEG,Cheng22_Cell;our analysis,A>C>B,screened,NaN
2,Grb14,Grb14,L2/3 subtypes,Cheng22_Cell,A>C>B,*,NaN
3,Adamts17,Adamts17,L2/3 subtypes,Cheng22_Cell,A>C>B,*,NaN
4,Ldb2,Ldb2,L2/3 subtypes,Cheng22_Cell,A>C>B,*,NaN
...,...,...,...,...,...,...,...
827,Vcl,Vcl,NRvsDR_DEG,our analysis,Endo,DRup,NaN
828,Zbtb38,Zbtb38,NRvsDR_DEG,our analysis,Astro,DRup,NaN
829,Zdbf2,Zdbf2,NRvsDR_DEG,our analysis,"L2/3,L4,L5IT,L6CT,L6IT","DRup,DRup,DRup,DRup,DRup",NaN
830,Zfp366,Zfp366,NRvsDR_DEG,our analysis,Endo,DRdn,NaN


In [13]:
pd.set_option('display.max_rows', 100)
df.groupby(['why included', 'source']).size().to_frame('number')

number
why included                              source                                                    
All cell types                            Chen22_biorxiv_Zador                                    48
                                          Chen22_biorxiv_Zador;Cheng22_Cell                       11
                                          Chen22_biorxiv_Zador;our analysis                        7
                                          Cheng22_Cell                                            51
                                          our analysis                                            38
All cell types;IEG;NRvsDR_DEG             Chen22_biorxiv_Zador;Hrvatin17_NatNeuro;our ana...       1
All cell types;L2/3 dynamic               Chen22_biorxiv_Zador;Cheng22_Cell;our analysis           2
                                          Cheng22_Cell;our analysis                                1
                                          our analysis                                             3
All cell types;L2/3 dynamic;NRvsDR_DEG    Chen22_biorxiv_Zador;our analysis                        1
                                          Cheng22_Cell;our analysis                                1
                                          our analysis                                             1
All cell types;L2/3 subtypes              Chen22_biorxiv_Zador;Cheng22_Cell                       11
All cell types;L2/3 subtypes;L2/3 dynamic Chen22_biorxiv_Zador;Cheng22_Cell;our analysis           5
All cell types;L2/3 subtypes;NRvsDR_DEG   Chen22_biorxiv_Zador;Cheng22_Cell;our analysis           1
All cell types;L2/3/4 subtypes at P14     Chen22_biorxiv_Zador;our analysis                        1
All cell types;L4 subtypes                Chen22_biorxiv_Zador;Cheng22_Cell                       11
All cell types;L4 subtypes;L2/3 dynamic   Chen22_biorxiv_Zador;Cheng22_Cell;our analysis           2
All cell types;L4/5 subtypes              Chen22_biorxiv_Zador;Hrvatin17_NatNeuro                  1
All cell types;L5 IT subtypes             Cheng22_Cell;Tasic18_Nature                              1
All cell types;NRvsDR_DEG                 Chen22_biorxiv_Zador;Cheng22_Cell;our analysis           1
                                          Chen22_biorxiv_Zador;our analysis                        3
                                          Cheng22_Cell;our analysis                                1
                                          our analysis                                             1
All cell types;V1_HVA_Spatial_Gradient    Chen22_biorxiv_Zador;Chou13_Science                      1
Astrocyte_NRvsDR_DEG                      Cheng22_Cell                                             6
                                          Cheng22_Cell;our analysis                                1
Astrocytes                                Bayraktar20_NatNeuro                                    40
Astrocytes;L2/3 dynamic                   Bayraktar20_NatNeuro;our analysis                        2
Astrocytes;NRvsDR_DEG                     Bayraktar20_NatNeuro;our analysis                        2
Early on marker                           Cheng22_Cell                                             3
Early on marker;L2/3 dynamic              Cheng22_Cell;our analysis                                1
IEG                                       Hrvatin17_NatNeuro                                      11
IEG;L2/3 dynamic                          Hrvatin17_NatNeuro;our analysis                          3
IEG;L2/3 dynamic;NRvsDR_DEG               Hrvatin17_NatNeuro;our analysis                          1
IEG;NRvsDR_DEG                            Hrvatin17_NatNeuro;our analysis                          6
L2/3 dynamic                              our analysis                                            96
L2/3 dynamic;NRvsDR_DEG                   our analysis                                             8
L2/3 subtypes                             Cheng22_Cell

In [6]:
df.groupby(['why included']).size().to_frame('number')

,number
why included,
All cell types,155
All cell types;IEG;NRvsDR_DEG,1
All cell types;L2/3 dynamic,6
All cell types;L2/3 dynamic;NRvsDR_DEG,3
All cell types;L2/3 subtypes,11
All cell types;L2/3 subtypes;L2/3 dynamic,5
All cell types;L2/3 subtypes;NRvsDR_DEG,1
All cell types;L2/3/4 subtypes at P14,1
All cell types;L4 subtypes,11


In [7]:
df.groupby(['source']).size().to_frame('number')

,number
source,
Bayraktar20_NatNeuro,40
Bayraktar20_NatNeuro;our analysis,4
Berg21_Nature,3
Buchanan22_PNAS_Allen,20
Buchanan22_PNAS_Allen;our analysis,1
Chen22_biorxiv_Zador,48
Chen22_biorxiv_Zador;Cheng22_Cell,33
Chen22_biorxiv_Zador;Cheng22_Cell;our analysis,11
Chen22_biorxiv_Zador;Chou13_Science,1


In [8]:
cond1 = df['why included'].str.contains("All cell types")
cond2 = df['source'].str.contains("Chen22_biorxiv_Zador")
print(df[cond1].shape)
print(df[cond2].shape)
print(df[cond1 & cond2].shape)

(205, 7)
(107, 7)
(107, 7)


In [15]:
"March3" in df['gene_name_data'].values

True

# Save

In [9]:
# fout = '../results/MERFISH_gene_panel_merged_Feb28.csv' 
# dfout.to_csv(fout, header=True, index=False)

In [10]:
# !head $fout

^C


In [12]:
# !wc -l $fout